<a href="https://colab.research.google.com/github/pengfei123xiao/ML_Basic/blob/master/models/Ch05-DecisionTree/RF_GBDT_XGBoost_LightGBM/GBDT_Key_Points.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GBDT Key Points
GBDT也是集成学习Boosting家族的成员，但是却和传统的Adaboost有很大的不同。回顾下Adaboost，是利用前一轮迭代弱学习器的误差率来更新训练集的权重，这样一轮轮的迭代下去。

GBDT也是迭代，使用了前向分布算法，但是**弱学习器只能使用CART回归树模型**，同时迭代思路和Adaboost也有所不同。

在GBDT的迭代中，假设我们前一轮迭代得到的强学习器是$f_{t−1}(x)$, 损失函数是$L(y, f_{t−1}(x))$, 我们本轮迭代的目标是找到一个CART回归树模型的弱学习器$h_t(x)$，让***本轮损失的损失***$L(y,f_t(x)=L(y, f_{t−1}(x)+h_t(x))$最小。

GBDT的思想可以用一个通俗的例子解释，假如有个人30岁，我们首先用20岁去拟合，发现损失有10岁，这时我们用6岁去拟合剩下的损失，发现差距还有4岁，第三轮我们用3岁拟合剩下的差距，差距就只有一岁了。如果我们的迭代轮数还没有完，可以继续迭代下面，每一轮迭代，拟合的岁数误差都会减小。



### 1. 前向分布算法
加法模型，从前向后，每步只学习一个基函数及其系数，逐步逼近目标。

### 2. 负梯度拟合
用损失函数的负梯度来拟合本轮损失的近似值，进而拟合一个CART回归树。通过损失函数的负梯度来拟合，找到了一种通用的拟合损失误差的办法，这样无论是分类问题还是回归问题，我们通过其损失函数的负梯度的拟合，就可以用GBDT来解决我们的分类回归问题。区别仅仅在于损失函数不同导致的负梯度不同而已。

第$t$轮的第$i$个样本的损失函数的负梯度表示为:
$r_{ti}=-[\frac{\partial L(y, f(x_i))}{\partial f(x_i)}]_{f(x)=f_{t-1}(x)}$

利用$(x_i,r_{ti})(i=1,2,..m)$, 我们可以拟合一颗CART回归树，得到了第$t$颗回归树，其对应的叶节点区域$R_{t_j,j}=1,2,...,J$。其中$J$为叶子节点的个数。

针对每一个叶子节点里的样本，我们求出使损失函数最小，也就是拟合叶子节点最好的的输出值$c_{t_j}$如下：

### 3. 损失函数
分类问题：deviance, exponential

回归问题：least square loss, huber loss, quantile loss


### 4. 回归
回归问题可以根据回归问题的损失函数解决，每次训练1棵决策树

Input: 最大迭代次数$T$, 损失函数$L$，训练样本集$T=\{(x_1, y_1), (x_2, y_2),...(x_m, y_m)\}$

Output: 强学习器$f(x)$

### 5. 二分类，多分类
GBDT的分类算法从思想上和GBDT的回归算法没有区别，但是由于样本输出不是连续的值，而是离散的类别，导致我们无法直接从输出类别去拟合类别输出的误差。

为了解决这个问题，主要有两个方法，一个是用指数损失函数，此时GBDT退化为Adaboost算法。另一种方法是用类似于逻辑回归的对数似然损失函数的方法。也就是说，我们用的是类别的预测概率值和真实概率值的差来拟合损失。本文仅讨论用对数似然损失函数的GBDT分类。而对于对数似然损失函数，我们又有二元分类和多元分类的区别。

* **二元分类算法**

对于二元GBDT，如果用类似于逻辑回归的对数似然损失函数，则损失函数为：

* **多元分类算法**

多元GBDT要比二元GBDT复杂一些，对应的是多元逻辑回归和二元逻辑回归的复杂度差别。假设类别数为K，则此时我们的对数似然损失函数为：

除了负梯度计算和叶子节点的最佳残差拟合的线性搜索，多元GBDT分类和二元GBDT分类以及GBDT回归算法过程相同。

### 6. 正则化

对GBDT进行正则化是为了防止过拟合。GBDT的正则化主要有三种方式。

1. 采用和Adaboost类似的正则化项，即步长(learning rate)。定义为$ν$,对于前面的弱学习器的迭代: $f_k(x)=f_{k-1}(x)+h_k(x)$
如果我们加上了正则化项，则有: $f_k(x)=f_{k-1}(x)+vh_k(x)$
$ν$的取值范围为$0<ν≤1$。对于同样的训练集学习效果，较小的ν意味着我们需要更多的弱学习器的迭代次数。通常我们用步长和迭代最大次数一起来决定算法的拟合效果。

2. 正则化的方式是通过子采样比例（subsample）。取值为(0,1]。注意这里的子采样和随机森林不一样，随机森林使用的是放回抽样，而这里是不放回抽样。如果取值为1，则全部样本都使用，等于没有使用子采样。如果取值小于1，则只有一部分样本会去做GBDT的决策树拟合。选择小于1的比例可以减少方差，即防止过拟合，但是会增加样本拟合的偏差，因此取值不能太低。推荐在[0.5, 0.8]之间。使用了子采样的GBDT有时也称作随机梯度提升树(Stochastic Gradient Boosting Tree, SGBT)。由于使用了子采样，程序可以通过采样分发到不同的任务去做boosting的迭代过程，最后形成新树，从而减少弱学习器难以并行学习的弱点。

 

3. 对弱学习器即CART回归树进行正则化剪枝。

### 7. 优缺点

**优点**

1) 可以灵活处理各种类型的数据，包括连续值和离散值。

2) 在相对少的调参时间情况下，预测的准确率也可以比较高。这个是相对SVM来说的。

3）使用一些健壮的损失函数，对异常值的鲁棒性非常强。比如 Huber损失函数和Quantile损失函数。

**缺点**

1) 由于弱学习器之间存在依赖关系，难以并行训练数据。不过可以通过自采样的SGBT来达到部分并行


### 8. sklearn参数


In [0]:
class sklearn.ensemble.GradientBoostingClassifier(loss='deviance', learning_rate=0.1,
                                                  n_estimators=100, subsample=1.0, 
                                                  criterion='friedman_mse', 
                                                  min_samples_split=2, min_samples_leaf=1,
                                                  min_weight_fraction_leaf=0.0, 
                                                  max_depth=3, min_impurity_decrease=0.0,
                                                  min_impurity_split=None, init=None, 
                                                  random_state=None, max_features=None, 
                                                  verbose=0, max_leaf_nodes=None, 
                                                  warm_start=False, presort='auto', 
                                                  validation_fraction=0.1, 
                                                  n_iter_no_change=None, tol=0.0001)


`loss`：损失函数

`learning_rate`：学习率

`n_estimators`：基础分类器数量

`subsample`：训练基础分类器样本比例

`criterion`：划分衡量指标

`min_samples_split`：决策树叶结点继续分裂最小样本数量

`min_samples_leaf`：决策树叶结点最小样本数量

`min_weight_fraction_leaf`：决策树叶结点最小加权样本数量

`max_depth`：决策树最大深度

`min_impurity_decrease`：决策树叶结点最小衡量指标提升

`init`：初始化方法

`random_state`：随机种子

`max_features`：搜索划分时考虑的特征数量

`verbose`：控制输出

`max_leaf_nodes`：决策树最大叶结点数量

`warm_start`：是否使用之前的输出

`presort`：是否尝试预先排序数据

`validation_fraction`：验证集比例，控制算法是否提前结束

`n_iter_no_change`：控制算法是否提前结束

`tol`：控制算法是否提前结束

### 9. 应用场景
1. Facebook使用其来自动发现有效的特征、特征组合，来作为LR模型中的特征，以提高 CTR预估（Click-Through Rate Prediction）的准确性，详见Practical Lessons from Predicting Clicks on Ads at Facebook。

2. GBDT在淘宝的搜索及预测业务上也发挥了重要作用。淘宝搜索/推荐系统背后深度强化学习与自适应在线学习的实践之路

3. 美团也有相关的实践：即时配送的ETA问题之亿级样本特征构造实践

**参考**：

西瓜书
          
cs229吴恩达机器学习课程
           
李航统计学习
           
https://ask.hellobi.com/blog/guodongwei1991/10501

公式推导参考：http://t.cn/EJ4F9Q0